In [1]:
import sys
sys.path.append('../')
from modules.shrec import utils, database_objects_views, comparison_class, comparison_class_test, query_objects_views, load_utils
from modules.shrec import shrec2019utils
import numpy as np
import matplotlib.pyplot as plt
import os


C:\Users\s161416\AppData\Local\Continuum\anaconda3\envs\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load test
Load the renders from query 3D models and the query images. 

In [2]:
main_directory = os.path.dirname(os.getcwd())
dataset_directory = os.path.join(main_directory, "dataset")
path_class_files = os.path.join(dataset_directory, "class_files")

# Paths to the corresponding images folder
renders_path = os.path.join(dataset_directory,"test_renders_64")
photos_path = os.path.join(dataset_directory,"test_images_64")


images, models = shrec2019utils.lossy_load_renders(renders_path)
print("Rendered images shape {}".format(images.shape))

images_photographs_test, name_photographs_test = shrec2019utils.lossy_load_photographs(photos_path, '.png')
print("Test photographs shape {}".format(images_photographs_test.shape))

Rendered images shape (11700, 64, 64, 3)


## Load VAE

In [6]:

vae_path = os.path.join(main_directory, "trained_models","simple_submission")  


# Load the parameters of the CVAE saved in a csv file
pd = load_utils.retrieve_parameters_dataframe(vae_path)
print(pd)


for num_model in range(len(pd["timestamp"])):
    print("Saving model", num_model)
    
    # Load the CVAE
    vae = load_utils.load_trained_vae(vae_path,num_model)
    
    type_layers = pd["type_layers"][num_model] # either convolutional or dense
    model_name = pd["timestamp"][num_model] # name given to a trained model 
    image_shape = eval(pd["image_shape"][num_model]) # shape of the images input to the model

    # Reformat image array
    if type_layers == "dense":
        renders = utils.flatten_normalize_images(images[:,:,:,:image_shape[2]])
    elif type_layers =="convolutional":
        renders = utils.normalize_images(images[:,:,:,:image_shape[2]])
    print("Flat array shape {}".format(images.shape))

    if type_layers== "dense":
        photographs_test = utils.flatten_normalize_images(images_photographs_test[:,:,:,:image_shape[2]])
    elif type_layers == "convolutional":
        photographs_test = utils.normalize_images(images_photographs_test[:,:,:,:image_shape[2]])
    print("Photographs flat test shape {}".format(images_photographs_test.shape))

    # Make the comparison between the query image and the 3D model database 
    database = database_objects_views.DatabaseObjectsViews(renders, models, models, vae)
    query_test = query_objects_views.QueryObjectsViews(photographs_test, name_photographs_test,name_photographs_test, vae)
    comparison_test = comparison_class_test.ComparisonClassTest(query_test, database, kl_distance_type = "categorical")
    
    
    # Save submission file of competition
    save_dir = os.path.join(main_directory, "submissions")
    method_name = model_name
    comparison_test.create_submission_file(save_dir, method_name)


           timestamp  double_convoltion  image_shape  intermediate_dim  \
0  2019-03-07-18-10_              False  (64, 64, 3)                64   

   kernel_size  latent_dim  num_classes  num_decoding_layers  \
0            3          50           30                    5   

   num_encoding_layers  r_loss  stride    type_layers  var_x type  batch_size  \
0                    2  binary       2  convolutional    1.0  VGG          32   

   epochs  
0     100  
Saving model 0
{'image_shape': (64, 64, 3), 'num_classes': 30, 'r_loss': 'binary', 'intermediate_dim': 64, 'type_layers': 'convolutional', 'num_encoding_layers': 2, 'num_decoding_layers': 5, 'kernel_size': 3, 'stride': 2, 'latent_dim': 50}
Reconstruction loss is binary cross entropy
Reconstruction loss is binary cross entropy
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
vae_train